In [2]:
# db connection

import pymysql
from sqlalchemy import create_engine
import keyring
import platform
import numpy as np

user = 'root'
pw = keyring.get_password('macmini_db', user)
host = '192.168.219.106' if platform.system() == 'Windows' else '127.0.0.1'
port = 3306
db = 'stock'

# DATA COLUMNS

In [3]:
# base data
COLUMNS_STOCK_DATA = ['date', 'open', 'high', 'low', 'close', 'volume']
COLUMNS_TRAINING_DATA = ['open', 'high', 'low', 'close', 'volume', 'close_ma5', 'volume_ma5', 'close_ma5_ratio', 'volume_ma5_ratio',
       'open_close_ratio', 'open_prev_close_ratio', 'high_close_ratio',
       'low_close_ratio', 'close_prev_close_ratio', 'volume_prev_volume_ratio',
       'close_ma10', 'volume_ma10', 'close_ma10_ratio', 'volume_ma10_ratio',
       'close_ma20', 'volume_ma20', 'close_ma20_ratio', 'volume_ma20_ratio',
       'close_ma60', 'volume_ma60', 'close_ma60_ratio', 'volume_ma60_ratio',
       'close_ma120', 'volume_ma120', 'close_ma120_ratio',
       'volume_ma120_ratio', 'close_ma240', 'volume_ma240',
       'close_ma240_ratio', 'volume_ma240_ratio', 'upper_bb',
       'lower_bb', 'bb_pb', 'bb_width', 'macd',
       'macd_signal', 'macd_oscillator', 'rs', 'rsi']

## Device

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# UTILITIES

## Get stock data 

In [5]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine


# get us stock price of a specific ticker
def get_stock_data(ticker, fro=None, to=None):

    # connect DB
    engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}:{port}/{db}')

    con = pymysql.connect(
        user=user,
        passwd=pw,
        host=host,
        db=db,
        charset='utf8'
    )
            
    mycursor = con.cursor()
    
    if fro is not None:
        if to is not None:               
            query = f""" 
                    SELECT * FROM price_global
                    WHERE ticker = '{ticker}'
                    AND date BETWEEN '{fro}' AND '{to}' 
                    """
        else:
            query = f""" 
                    SELECT * FROM price_global
                    WHERE ticker = '{ticker}'
                    AND date >= '{fro}'
                    """
    
    else:
        if to is not None:
            query = f""" 
                    SELECT * FROM price_global
                    WHERE ticker = '{ticker}'
                    AND date <= '{to}' 
                    """
        else:
            query = f""" 
                    SELECT * FROM price_global
                    WHERE ticker = '{ticker}'
                    """
            
    print(query)
    prices = pd.read_sql(query, con=engine)
    con.close()
    engine.dispose()
    return prices

### Sample code

In [6]:
stock_code = 'AAPL'
fro = '2010-01-01'
to = '2020-12-31'
df = get_stock_data(stock_code, fro=fro, to=to)

 
                    SELECT * FROM price_global
                    WHERE ticker = 'AAPL'
                    AND date BETWEEN '2010-01-01' AND '2020-12-31' 
                    


## Functions

### Sigmoid function

In [7]:
def sigmoid(x):
    x = max(min(x, 10), -10)
    return 1. / (1. + np.exp(-x))

## Preprocessing

In [8]:
def preprocess(data):
    
    # moving average
    windows = [5, 10, 20, 60, 120, 240]
    for window in windows:
        data[f'close_ma{window}'] = data['close'].rolling(window).mean()
        data[f'volume_ma{window}'] = data['volume'].rolling(window).mean()
        data[f'close_ma{window}_ratio'] = (data['close'] - data[f'close_ma{window}']) / data[f'close_ma{window}']
        data[f'volume_ma{window}_ratio'] = (data['volume'] - data[f'volume_ma{window}']) / data[f'volume_ma{window}']
        data['open_close_ratio'] = (data['open'].values - data['close'].values) / data['close'].values
        data['open_prev_close_ratio'] = np.zeros(len(data))
        data.loc[1:, 'open_prev_close_ratio'] = (data['open'][1:].values - data['close'][:-1].values) / data['close'][:-1].values
        data['high_close_ratio'] = (data['high'].values - data['close'].values) / data['close'].values
        data['low_close_ratio'] = (data['low'].values - data['close'].values) / data['close'].values
        data['close_prev_close_ratio'] = np.zeros(len(data))
        data.loc[1:, 'close_prev_close_ratio'] = (data['close'][1:].values - data['close'][:-1].values) / data['close'][:-1].values 
        data['volume_prev_volume_ratio'] = np.zeros(len(data))
        data.loc[1:, 'volume_prev_volume_ratio'] = (
            # if volume is 0, change it into non zero value exploring previous volume continuously
            (data['volume'][1:].values - data['volume'][:-1].values) / data['volume'][:-1].replace(to_replace=0, method='ffill').replace(to_replace=0, method='bfill').values
        )
    
    # Bollinger band
    data['middle_bb'] = data['close'].rolling(20).mean()
    data['upper_bb'] = data['middle_bb'] + 2 * data['close'].rolling(20).std()
    data['lower_bb'] = data['middle_bb'] - 2 * data['close'].rolling(20).std()
    data['bb_pb'] = (data['close'] - data['lower_bb']) / (data['upper_bb'] - data['lower_bb'])
    data['bb_width'] = (data['upper_bb'] - data['lower_bb']) / data['middle_bb']
    
    # MACD
    macd_short, macd_long, macd_signal = 12, 26, 9
    data['ema_short'] = data['close'].ewm(macd_short).mean()
    data['ema_long'] = data['close'].ewm(macd_long).mean()
    data['macd'] = data['ema_short'] - data['ema_long']
    data['macd_signal'] = data['macd'].ewm(macd_signal).mean()
    data['macd_oscillator'] = data['macd'] - data['macd_signal']
    
    # RSI
    data['close_change'] = data['close'].diff()
    # data['close_up'] = np.where(data['close_change'] >=0, df['close_change'], 0)
    data['close_up'] = data['close_change'].apply(lambda x: x if x >= 0 else 0)
    # data['close_down'] = np.where(data['close_change'] < 0, df['close_change'].abs(), 0)
    data['close_down'] = data['close_change'].apply(lambda x: -x if x < 0 else 0)
    data['rs'] = data['close_up'].ewm(alpha=1/14, min_periods=14).mean() / data['close_down'].ewm(alpha=1/14, min_periods=14).mean()
    data['rsi'] = 100 - (100 / (1 + data['rs']))
    
    
    return data

### Sample code

In [9]:
df_adj = preprocess(df)

## Load data

load_data() function is a combined function for getting data from databases and preprocessing it into training data.

In [10]:
def load_data(stock_code, fro, to):
    ''' 
    Arguments
    ----------
    - stock_code : unique stock code
    - fro : start date
    - to : end data
    
    Returns
    --------
    df_adj : entire prerprocessed data
    stock_data : data for plotting chart
    training_data : data for training a model
    '''
    
    df = get_stock_data(stock_code, fro, to)
    df_adj = preprocess(df).dropna().reset_index(drop=True)
    # df_adj.dropna(inplace=True).reset_index(drop=True)
    
    stock_data = df_adj[COLUMNS_STOCK_DATA]
    training_data = df_adj[COLUMNS_TRAINING_DATA]
    
    return df_adj, stock_data, training_data.values

### Sample code

In [11]:
df_adj, stock_data, training_data = load_data(stock_code, fro, to)

 
                    SELECT * FROM price_global
                    WHERE ticker = 'AAPL'
                    AND date BETWEEN '2010-01-01' AND '2020-12-31' 
                    


# Environment

In [12]:
# environment

import numpy as np
import pandas as pd

# environment

class Environment:
    ''' 
    Attribute
    ---------
    - stock_data : stock price data such as 'open', 'close', 'high', 'low', 'volume'
    - state : current state
    - idx : current postion of stock data
    
    
    Functions
    --------
    - reset() : initialize idx and state
    - observe() : move idx into next postion and get a new state
    - get_price() : get close price of current state
    - get_state() : get current state
    '''
    
    def __init__(self, stock_data=None):
        self.close_price_idx = 4    # index postion of close price
        self.open_price_idx = 1     # index position of open price
        self.stock_data = stock_data
        self.state = None
        self.idx = -1
        
    def reset(self):
        self.state = None
        self.idx = -1
        
    def observe(self):
        # move to next day and get price data
        # if there is no more idx, return None
        if len(self.stock_data) > self.idx + 1:
            self.idx += 1
            self.state = self.stock_data.iloc[self.idx]
            return self.state
        return None
    
    def get_close_price(self):
        # return close price
        if self.state is not None:
            return self.state[self.close_price_idx]
        return None
    
    def get_state(self):
        # return current state
        if self.state is not None:
            return self.state
        return None
        

### Sample code

In [13]:
env = Environment(stock_data)

In [14]:
env.reset()

In [15]:
env.observe()

date      2010-12-14
open       11.392857
high       11.490357
low        11.519286
close      11.438929
volume      9.684193
Name: 0, dtype: object

In [16]:
env.get_state()

date      2010-12-14
open       11.392857
high       11.490357
low        11.519286
close      11.438929
volume      9.684193
Name: 0, dtype: object

In [17]:
env.observe()

date      2010-12-15
open       11.399643
high       11.428571
low        11.535714
close      11.441429
volume      9.686307
Name: 1, dtype: object

In [18]:
env.get_state()['close']

11.441429138183594

In [19]:
env.observe()

date      2010-12-16
open       11.432143
high         11.4675
low        11.521786
close      11.473214
volume      9.713216
Name: 2, dtype: object

In [21]:
env.get_state()['open']

11.432143211364746

# Agent

In [22]:
class Agent:
    ''' 
    Attributes
    --------
    - enviroment : instance of environment
    - initial_balance : initial capital balance
    - min_trading_price : minimum trading price
    - max_trading_price : maximum trading price
    - balance : cash balance
    - num_stocks : obtained stocks
    - portfolio_value : value of portfolios (balance + price * num_stocks)
    - num_buy : number of buying
    - num_sell : number of selling
    - num_hold : number of holding
    - ratio_hold : ratio of holding stocks
    - profitloss : current profit or loss
    - avg_buy_price_ratio : the ratio average price of a stock bought to the current price
    
    Functions
    --------
    - reset() : initialize an agent
    - set_balance() : initialize balance
    - get_states() : get the state of an agent
    - decide_action() : exploration or exploitation behavior according to the policy net
    - validate_action() : validate actions
    - decide_trading_unit() : decide how many stocks are sold or bought
    - act() : act the actions
    '''
    
    def __init__(self,
                 env,
                 initial_balance=100000, min_trading_price=10, max_trading_price=10000):      
        
        # agent state dimensions
        ## (ratio_hold, profit-loss ratio, current price to avg_buy_price ratio)
        # self.state_dim = 3
        
        # trading charge and tax
        self.TRADING_CHARGE = 0.00015    # trading charge 0.015%
        self.TRADING_TAX = 0.002          # trading tax = 0.2%
        
        # action space
        self.ACTION_BUY = 0      # buy
        self.ACTION_SELL = 1     # sell
        self.ACTION_HOLD = 2     # hold
        
        # get probabilities from neural nets
        self.ACTIONS = [self.ACTION_BUY, self.ACTION_SELL, self.ACTION_HOLD]
        self.NUM_ACTIONS = len(self.ACTIONS)      # output number from nueral nets
        
        
        # get current price from the environment
        self.env = env
        self.initial_balance = initial_balance
        self.done = False
        
        # minumum and maximum trainding price
        self.min_trading_price = min_trading_price
        self.max_trading_price = max_trading_price
        
        # attributes for an agent class
        self.balance = initial_balance
        self.num_stocks = 0
        
        # value of portfolio : balance + num_stocks * {current stock price}
        self.portfolio_value = self.balance
        self.num_buy = 0
        self.num_sell = 0
        self.num_hold = 0
        
        # three states of Agent class
        self.ratio_hold = 0
        self.profitloss = 0
        self.avg_buy_price = 0
        
    def reset(self):
        self.balance = self.initial_balance
        self.num_stocks = 0
        self.portfolio_value = self.balance
        self.num_buy = 0
        self.num_sell = 0
        self.num_hold = 0
        self.ratio_hold = 0
        self.profitloss = 0
        self.avg_buy_price = 0
        self.done = False
        
    def set_initial_balance(self, balance):
        self.initial_balance = balance
        
    def get_states(self):
        # return current profitloss based on close price
        close_price = self.env.get_state()['close']
        self.portfolio_value = self.balance + close_price * self.num_stocks
        self.profitloss = self.portfolio_value / self.initial_balance - 1
        return self.profitloss
        
    def decide_action(self, pred_value, eps):
        # act randomly with epsilon probability, act according to neural network  with (1 - epsilon) probability
        confidence = 0
        
        # if theres is a pred_policy, follow it, otherwise follow a pred_value
        pred = pred_value
            
        # there is no prediction from both pred_policy and pred_value, explore!
        if pred is None:
            eps = 1
        else:
            maxpred = np.max(pred)
            # if values for actions are euqal, explore!
            if (pred == maxpred).all():
                eps = 1
        
            # if the diffrence between buying and selling prediction policy value is less than 0.05, explore!   
                    
        # decide whether exploration will be done or not
        if np.random.rand() < eps:
            exploration = True
            action = np.random.randint(self.NUM_ACTIONS) 
        else: 
            exploration = False
            action = np.argmax(pred)
            
        confidence = .5
        if pred_value is not None:
            confidence = sigmoid(pred[action])
            
        return action, confidence, exploration
    
    def validate_action(self, action, open_price):
        # validate if the action is available
        if action == self.ACTION_BUY:
            # check if al least one stock can be bought.
            if self.balance < open_price * (1 + self.TRADING_CHARGE):
                return False
        elif action == self.ACTION_SELL:
            # check if there is any sotck that can be sold
            if self.num_stocks <= 0:
                return False
        
        return True
    
    def decide_trading_unit(self, confidence):
        # adjust number of stocks for buying and selling according to confidence level
        if np.isnan(confidence):
            return self.min_trading_price
        
        # set buying price range between self.min_trading_price + added_trading_price [min_trading_price, max_trading_price]
        # in case that confidence > 1 causes the price over max_trading_price, we set min() so that the value cannot have larger value than self.max_trading_price - self.min_trading_price
        # in case that confidence < 0, we set max() so that added_trading_price cannot have negative value.
        added_trading_price = max(min(
            int(confidence * (self.max_trading_price - self.min_trading_price)),
            self.max_trading_price - self.min_trading_price
        ), 0)
        
        trading_price = self.min_trading_price + added_trading_price
        
        return max(int(trading_price / self.environment.get_price()), 1)
    
    def step(self, action, confidence):
        '''
        Arguments
        ---------
        - action : decided action from decide_action() method based on exploration or exploitation (0 or 1)
        - confidence : probabilitu from decide_action() method, the probability from policy network or the softmax probability from value network
        '''
        
        # get the next open price from the environment
        next_state = self.env.observ()
        
        if next_state is None:
            self.done = True
        else:
            self.done = False
        
        open_price = next_state['open']
        
        if not self.validate_action(action, open_price):
            action = self.ACTION_HOLD
        
        # buy
        if action == self.ACTION_BUY:
            # decide how many stocks will be bought
            trading_unit = self.decide_trading_unit(confidence)
            balance = (
                self.balance - open_price * (1 + self.TRADING_CHARGE) * trading_unit
            )
            
            # if lacks of balance, buy maximum units within the amount of money available
            if balance < 0:
                trading_unit = min(
                    int(self.balance / (open_price * (1 + self.TRADING_CHARGE))),
                    int(self.max_trading_price / open_price)
                )
                
            # total amount of money with trading charge
            invest_amount = open_price * (1 + self.TRADING_CHARGE) * trading_unit
            if invest_amount > 0:
                self.avg_buy_price = (self.avg_buy_price * self.num_stocks + open_price * trading_unit) / (self.num_stocks + trading_unit)
                self.balance -= invest_amount
                self.num_stocks += trading_unit
                self.num_buy += 1
                
        # sell
        elif action == self.ACTION_SELL:
            # decide how many stocks will be sold
            trading_unit = self.decide_trading_unit(confidence)
            
            # if lacks of stocks, sell maximum units available
            trading_unit = min(trading_unit, self.num_stocks)
            
            # selling amount
            invest_amount = open_price * (
                1 - (self.TRADING_TAX + self.TRADING_CHARGE)
            ) * trading_unit
            
            if invest_amount > 0:
                # update average buy price
                self.avg_buy_price = (self.avg_buy_price * self.num_stocks - open_price * trading_unit) / (self.num_stocks - trading_unit) if self.num_stocks > trading_unit else 0
                self.num_stocks -= trading_unit
                self.balance += invest_amount
                self.num_sell += 1
                
        # hold
        elif action == self.ACTION_HOLD:
            self.num_hold += 1
            
        # update portfolio value with close price
        close_price = next_state['close']
        
        self.portfolio_value = self.balance + close_price * self.num_stocks
        self.profitloss = self.portfolio_value / self.initial_balance - 1
        
        info = ''
        
        return self.next_state, self.profitloss, self.done, info             # (next_states, profitloss, done, info)
    
    

# Learner

## DQN Learner

### Method

##### Experience buffer

- We get transition data $(s,a,r,s^\prime)$, save into buffer, and train with uniform random sampling data from buffer.

- Q-learning is off-poilcy learning.

#### Target netwrok

- We set separate target network and update it periodically.

    - $\theta_i$ : training parameter for $i$th iteration

    - $\theta^-_i$ : target calculation network for $i$th iteration

    - $U(D)$ : replay memory of transitions $\rightarrow \pi_0, \cdots, \pi_i$ dataset.

#### Loss function

$$L_i(\theta_i)=E_{s,a,r,s^\prime}\left[(r+\gamma\max_{a^\prime}Q(s^\prime, a^\prime, Q^-_i)-Q(s,a;\theta_i))^2\right]$$

### Network architecture

- To reudce compuration complexity, neural network get states and return all action-values.

<img src='./image/3-s2.0-B9780323857871000117-f06-02-9780323857871.jpg'>

In [23]:
from collections import deque
import random

class ReplayMemory:
    
    def __init__(self, replay_capacity=480):
        self.buffer =deque([], maxlen=replay_capacity)
        
    def getsize(self):
        return len(self.buffer)
    
    def append(self, transition):
        self.buffer.append(transition)
        
    def sample(self, size):
        buffer_size = len(self.buffer)
        if buffer_size >= size:
            samples = random.sample(self.buffer, size)
        else:
            assert False, f"Buffer size ({buffer_size}) is smaller than the sample size ({size})"
        return samples

In [24]:
r = ReplayMemory()

In [25]:
r.buffer

deque([], maxlen=480)

In [27]:
# dqn_learner.py

import torch
from torch import nn 

class DQNAgent: 
    
    def __init__(self, 
                 env, agent,
                 stock_code=None, 
                 stock_data=None, training_data=None,
                 discount_factor=0.8, num_epochs=1000, eps_init=1, eps_final=0.05, eps_decrease_step=1000, batch_size=60, replay_init_ratio=0.3, replacy_capacity=480,
                 num_steps=1, lr=0.005, value_network=None, reuse_models=True, 
                 output_path='', gen_output=True):
        
        # environement paramters
        self.stock_code = stock_code
        self.stock_data = stock_data
        self.training_data = training_data
        
        self.env = Environment(stock_code=self.stock_code)
        self.agent = Agent(self.env)
        
        
        # reinforcement learning parameters
        self.discount_factor = discount_factor
        self.num_epochs = num_epochs
        self.eps_init = eps_init 
        self.eps_final = eps_final 
        self.eps_decrease_step = eps_decrease_step
        self.batch_size = batch_size
        self.replay_capacity = replacy_capacity
        self.replay_init_ratio = replay_init_ratio    
        
        # network
        self.lr = lr
        self.num_steps = num_steps
        self.value_network = value_network
        self.reuse_models = reuse_models 
        self.input_dim = (self.num_steps, self.training_data.shape[1])
        kernel_size = 2
        self.network = nn.Sequential(
            nn.BatchNorm2d(self.input_dim),
            nn.Conv2d(self.input_dim, 1, kernel_size),
            nn.BatchNorm2d(1),
            nn.Flatten(),
            nn.Dropout(p=0.1),
            nn.Linear(self.input_dim - (kernel_size - 1), 128),
            nn.BatchNorm2d(128),
            nn.Dropout(p=0.1),
            nn.Linear(128, 64),
            nn.BatchNorm2d(64),
            nn.Dropout(p=0.1),
            nn.Linear(64, 32),
            nn.BatchNorm2d(32),
            nn.Dropout(p=0.1),
            nn.Linear(32, self.NUM_ACTIONS)
        )
        self.target_network = nn.Sequential(
            nn.BatchNorm2d(self.input_dim),
            nn.Conv2d(self.input_dim, 1, kernel_size),
            nn.BatchNorm2d(1),
            nn.Flatten(),
            nn.Dropout(p=0.1),
            nn.Linear(self.input_dim - (kernel_size - 1), 128),
            nn.BatchNorm2d(128),
            nn.Dropout(p=0.1),
            nn.Linear(128, 64),
            nn.BatchNorm2d(64),
            nn.Dropout(p=0.1),
            nn.Linear(64, 32),
            nn.BatchNorm2d(32),
            nn.Dropout(p=0.1),
            nn.Linear(32, self.NUM_ACTIONS)
        )   
        
        # We do not train target_network
        for param in self.target_network.parameters():
            param.requires_grad = False    
        
        # memory
        self.replay_memory = ReplayMemory()
        self.memory_sample = []     # training data sample
        self.memory_action = []     # actions taken
        self.memory_reward = []     # reward obtained
        self.memory_value = []      # prediction value for action
        self.memory_pv = []         # portfolio value
        self.memory_num_stocks = [] # number of stocks
        self.memory_exp_idx = []    # exploration index
        
        # exploration epoch info
        self.loss = 0               # loss during epoch
        self.itr_cnt = 0            # number of iterations with profit
        self.exploration_cnt = 0    # count of exploration
        self.batch_size = 0         # number of training
        
        # log path
        self.output_path = output_path
        self.gen_output = gen_output
        
    def update_target_network(self):
        self.target_network.load_state_dict(self.net.state_dict())
        
    def set_optimizer(self):
        self.optimizer = torch.optim.NAdam(
            params=self.network.parameters(),
            lr=self.lr,
            weight_decay=1e-3
        )
        
    def forward(self, x):
        Qs = self.network(x)
        return Qs
    
    def forward_target_network(self, x):
        Qs = self.target_network(x)
        return Qs
    
    def get_argmax_action(self, x):
        # transform torch tensor with two dimentional matrix
        s = torch.from_numpy(x).reshape(1, -1).float()
        Qs = self.forward(s)
        # get item(integer) in argmax tensor
        argmax_action = Qs.argmax(dim=-1).item()
        return argmax_action
    
    def train(self):
        transitions = self.replay_memory.sample(self.batch_size)
        states, actions, rewards, next_states, dones = zip(*transitions)
        
        states_array = np.stack(states, axis=0)                     # (n_batch, states)
        actions_array = np.stack(actions, axis=0, dtype=np.int64)   # (n_batch)
        rewards_array = np.stack(rewards, axis=0)                   # (n_batch)
        next_states_array = np.stack(next_states, axis=0)           # (n_batch, states)
        dones_array = np.stack(dones, axis=0)                       # (n_batch)
        
        states_tensor = torch.from_numpy(states_array).float()
        actions_tensor = torch.from_numpy(actions_array)
        rewards_tensor = torch.from_numpy(rewards_array).float()
        next_states_tensor = torch.from_numpy(next_states_array).float()
        dones_tensor = torch.from_numpy(dones_array).float()
        
        Qs = self.forward(states_tensor)
        next_Qs = self.forward_target_network(next_states_tensor)
        
        chosen_Q = Qs.gather(dim=-1, index=actions_tensor.reshape(-1, 1)).reshape(-1)   # (n_batch, 1) -> (n_batch)
        target_Q = rewards_array + (1 - dones_tensor) * self.discount_factor * next_Qs.max(dim=-1).values
        
        # loss function
        criterion = nn.SmoothL1Loss()
        loss = criterion(chosen_Q, target_Q)
        
        # update by gradient descent
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss.item()
        
    def get_eps(self, step):
        eps_init = self.eps_init
        eps_final = self.eps_final
        if step >= self.eps_decrease_step:
            eps = eps_final
        else:
            m = (eps_final - eps_init) / self.eps_decrease_step
            eps = eps_init + m * eps
        return eps
        
        
        